In [1]:
import torch, torch_geometric
from torch_geometric.datasets import TUDataset
from sparse_gcn import SparseGCN
from torch_geometric.data import DataLoader

In [2]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
train_size = int(0.8 * len(dataset))
data_train, data_val = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

In [16]:
model = SparseGCN(dataset.num_features, [64] * 8, [dataset.num_classes], use_distance_kernel=False)
model = model.to('cuda')

In [17]:
print(model.parameters)

<bound method Module.parameters of SparseGCN(
  (convs): ModuleList(
    (0): GCNResidualBlock(
      (conv1): GCNConv(3, 64)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (conv2): GCNConv(64, 64)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): GCNResidualBlock(
      (conv1): GCNConv(64, 64)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (conv2): GCNConv(64, 64)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (2): GCNResidualBlock(
      (conv1): GCNConv(64, 64)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (conv2): GCNConv(64, 64

In [18]:
sum(p.numel() for p in model.parameters())

65094

In [19]:
loader_train = DataLoader(data_train, batch_size=64, shuffle=True)
loader_val = DataLoader(data_val, batch_size=64, shuffle=True)

In [22]:
optimizer = torch.optim.Adamax(model.parameters(), lr=1e-3)
loss_function = torch.nn.functional.cross_entropy

In [23]:
for epoch in range(20000):
    #print(f'Starting epoch {epoch}')
    epoch_loss = 0
    model.train()
    for idx, data in enumerate(loader_train):
        optimizer.zero_grad()
        outputs = model(data.to('cuda'))
        loss = loss_function(outputs, data.y.to('cuda'))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    model.eval()
    total, correct = 0, 0
    for idx, data in enumerate(loader_val):
        outputs = model(data.to('cuda'))
        _, predictions = torch.max(outputs.data, 1)
        total += data.y.size(0)
        correct += (predictions == data.y.to('cuda')).sum().item()
    print(f'\rVal Accuracy {correct / total:4f}, Train loss {epoch_loss / len(loader_train):4f}', end='\r')
    
    #print(f'\rTrain loss {epoch_loss / len(loader_train):4f}', end='\r')
        
        
        
        
        

KeyboardInterrupt: 